
# NO MACHINE LEARNING LIBARY USED FOR TRAINING/REGRESSION/CLASSFICATION
# sklean.feature_extraction ONLY USED for language data into Vector transformation
# preprocess dataset

In [1]:
import pandas as pd
import csv
import codecs
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from sklearn.feature_extraction.text import CountVectorizer#ONLY USED FOR TRANSFORM FROM LANGUAGE TO VECTOR
from numpy import linalg as LA
import math

In [2]:
df = pd.read_csv('./Tab.delimited.Cleaned.dataset.WITH.variable.labels.csv', sep='\t',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df3 = pd.read_csv('./ML3AllSites.csv', sep=',',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
class NeuralNetwork:
    def __init__(self, n_h, n_iterate=10, learning_rate=1, dic = []):
        self.n_x = None  # size of the input layer
        self.n_h = n_h  # size of the hidden layer
        self.n_y = None # size of the output layer
        self.W1 = None
        self.W2 = None
        self.b1 = None
        self.b2 = None
        self.A1 = None
        self.A2 = None  # sigmoid output of the second activation
        self.learning_rate = learning_rate
        self.n_iterate = n_iterate
        self.lamda = 0.01
#         self.dic = dic
        self.dic = dic
    
    def initialize_parameters(self):
        self.W1 = np.random.randn(self.n_h, self.n_x) * 0.01
        self.b1 = np.zeros((self.n_h, 1))
        self.W2 = np.random.randn(self.n_y, self.n_h) * 0.01
        self.b2 = np.zeros((self.n_y, 1))
       
    def MSE(self,Y):
        cost = 0
        for i in range(len(Y)):
            for j in range(len(Y[0])):
                cost+=(Y[i][j]-self.A2[i][j])**2
        cost = cost/len(Y)
        return cost
        
    def relu(self, z):
        return z * (z > 0)
    
#     def softmax(self,z):
#         exps = np.exp(z-np.max(z,axis= 1,keepdims = True))
#         return exps/np.sum(exps,axis = 1, keepdims = True)
    
    

#     def softmax(self,A):  
#         expA = np.exp(A)
#         return expA / expA.sum(axis=1, keepdims=True)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    
    def cro_entro(self,Y):
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean()
        return cost
    
    #changed for mult layer
    #add lasso regularization 
    def compute_cost(self, Y):# cross entrophy
        #cost = np.linalg.norm(self.A2 - Y)
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean() 
#         print('shape cost is: ' ,cost.shape)

#         <----lasso regularization ---->
#         tmpcost = self.A2.T
#         tmpcost = np.sum(tmpcost,axis = 1)/len(Y)
#         <----End of lasso regularization implementation ----- >
        return np.squeeze(cost)

    def forward_propagation(self, X):
        self.A1 = self.relu(self.W1 @ X + self.b1)
        #dropout function for output in hidden layer 
        u1 = np.random.binomial(1,0.5,size = self.A1.shape)
        self.A1 *= u1
    
        self.A2 = self.sigmoid(self.W2 @ self.A1 + self.b2)

#         u2 = np.random.binomial(1,0.5,size = self.A2.shape)
#         self.A2 *= u2
#         self.A2 = self.soft_max_For_multi_attribute(self.A2,self.dic)

        
    
    def backward_propagation(self, X, Y):
        m = X.shape[1]
#       <------dz2 for lasso regression ----- > 
#         dZ2 = (self.A2 - Y)/m  +self.lamda/m
#       <------dz2 for lasso regression ----- > 

        dZ2 = (self.A2 - Y)/m
        dW2 = dZ2 @ self.A1.T 
        db2 = np.sum(dZ2, axis=1, keepdims=True) 
        dZ1 = self.W2.T @ dZ2 * (self.A1 > 0)
        dW1 = dZ1 @ X.T 
        db1 = np.sum(dZ1, axis=1, keepdims=True) 

        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def fit(self, X, Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
        print('n_x is',self.n_x ,'n_y is ',self.n_y )
        self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                  self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))

    def fit_continue(self,X,Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
#         if self.W1 is None:
#             self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                 self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))
                
    def predict(self, X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)

        
        A2 = self.sigmoid(self.W2 @ A1 + self.b2)
        tmp = np.dot(self.W2,self.A1)
    
        return A2.T
    
    def predict_one_layer(self,X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)
        return A1.T
    
    def soft_max_For_multi_attribute(self,arr,dic):
        #create array with all dict keys based on insertion order
        tmp_arr = np.copy(arr)
        
        dicarr = list(dic.keys())
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length    
            #find the targeted array need to perform softmax
            #create a dic to contain the location of those onehotencoding attribute
            loc_dic = []
            for i in range(cur_dict_length-1):
                loc_dic.append(cur_loc+i)
            #find targed array
            target = tmp_arr[:,loc_dic]
            #get softmaxed arr
            arr_soft_max = self.api_softmax(target)
            #reassig value back to orignal arr
            tmp_arr[:,loc_dic] = arr_soft_max
            #update cur_loc
            cur_loc +=cur_dict_length
            
        return tmp_arr

    def api_softmax(self,z):
        exps = np.exp(z)
        return exps/np.sum(exps,axis = 1, keepdims = True)

In [5]:
class KMeans:
    def __init__(self, n_clusters=64):
        self.n_clusters = n_clusters  # number of clusters
        self.centers = None  # to record the centers
        self.labels = None
        self.Y = None

    def random_center(self,Y): #only for 1d dataset
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
            
            
    def random_center2(self,Y):
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y[0]))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
#         for j in range(0):
#             Y_j_min = self.Y[:,j].min()
#             Y_j_max = self.Y[:,j].max()
#             self.centers[:,j] = Y_j_min + (Y_j_max - Y_j_min) * self.centers[:,j]
            
            
            
    def dist(self, point1, point2): #old one
        return 2*(point1[0]-point2[0])**2 + 4*(point1[1]-point2[1])**2 + 3*(point1[2]-point2[2])**2

    def dist2(self,point1,point2):
        return LA.norm(point1-point2)**2
    
    def fit(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    def fit2(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center2(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    
    def transform(self, Y):
        out = np.zeros(Y.shape)
        for i in range(self.n_clusters):
            out[self.labels==i] = self.centers[i]
        return out
    

# oneHOTENCODING

In [6]:
def oneHotEncoding(Y, kmeans):
    out = np.zeros((len(Y), kmeans.n_clusters))
    for i in range(len(Y)):
        out[i, Y[i]] = 1
    return out

# test clustering numbered data:
    step1 : get single column
    step2 : sort in increasing order
    step3 : based on distribution, cut data into 4 section with equally number of candidate
    step4 : if data is null, set isnon to be true for that column
    step5 : append new datacol to input_arr

In [7]:
def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def getdistriarr(df,arr_str):
    arr = df[arr_str].values
    di = []
    di_og = []  #RETURN OG SINCE di will be sort
    for i in range(len(arr)):
        if isint(arr[i]) == True:
            di.append(int(arr[i]))
            di_og.append(int(arr[i]))
        else:
            di.append(-1)
            di_og.append(-1)
    di.sort()
    ans_arr = []
    leng = len(di)
    ans_arr.append(di[int(leng/3)])
#     ans_arr.append(di[int(leng/2)])
    ans_arr.append(di[int(leng*0.66)])
    ans_arr.append(di[int(leng-1)])
    return di_og,ans_arr

def oneHotEncoding_return_arr(att_arr,class_arr): #one 
    #add one more column for each row since last col used as flag
    choice = len(class_arr)
    out = np.zeros((len(att_arr),len(class_arr)+1))
    for i in range(len(att_arr)):
        loc = 0
        #first jude if data is null
        if att_arr[i] == -1:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            for j in range(len(class_arr)):
                if att_arr[i] == class_arr[j]:
                    out[i][j] = 1
                    break
        
    return out


def GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,attr_name,og_dic):
    att_arr,class_arr = getdistriarr(df,attr_name)
    dic_ans = create_dic_for_classification(class_arr)
    append_arr_of_dic_to_overall(og_dic,dic_ans,attr_name)

    return oneHotEncoding_return_arr(att_arr,class_arr)

def create_dic_for_classification(class_arr):
    cur_dic = {}
    total_len = len(class_arr)
    for i in range(len(class_arr)):
        cur_dic[class_arr[i]] = i
    return cur_dic

def append_arr_of_dic_to_overall(og,dic,att_name):
    og[att_name] = dic
    
    
def append_arr(old,new):
    #both old and new has the same num of row
    #create a new nparray
    
    newdim = len(old[0])+len(new[0])
    ans = np.zeros((len(old),newdim))
    print(ans.shape)
    for i in range(len(old)):
        ans[i] = np.append(old[i],new[i])
    return ans

# STEP TWO CONVERT NLP DATASET INTO VECTOR:
    STEP1: vectorize dataset
    STEP2: apply kmeans clustering to language data
    STEP3: ONEHOT ENCODING
    STEP4: APPEND CLASSIFICATION DATASET INTO old CLASSFICAITION ARR

In [8]:
def NLPDATAPRO(allsentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(allsentences)
    ans = X.toarray()
    return ans

def createtextarr_ONEHOT_ENCODING(attri_col): #ONEHOT_ENCODING
    df2 = df[attri_col]
    arr = df2.values
    for i in range(len(arr)):
        di.append(arr[i])
    return arr

# STEP THREE CONVERT TEXT_DATASET WITH FIXED classification into vector

In [9]:
def processnull(arr):
    arrnew = []
    for i in range(len(arr)):
        if type(arr[i]) == float and math.isnan(arr[i]) :
            arrnew.append('Nan')
        else:
            arrnew.append(arr[i])
    return arrnew

def returnuniqiearr(og):
    ar = np.asarray(og)
    return np.unique(ar)

# TODO: create a function to append text to old vecto(remember to append flag)

# TODO: TEST NN TONIGHT?

In [10]:
def create_onehoted_text_fixed_classification(attribu_name,df,km_num):
    df_cur = df[attribu_name]
    attribute_arr = df_cur.values
    
    arr_no_null = processnull(attribute_arr) #create new arr with format fit by NLP(CountVectorizer)
   
    NLP_ARR = NLPDATAPRO(arr_no_null)
    #BEFORE ONEHOT_ENCODING
    #APPLY KMEANS CLUSTERING SINCE TOO MANY DIFFERENT CLASSIFICATION
    KM_TMP = KMeans(km_num)
    KM_TMP.fit2(NLP_ARR)
    labels_arr = KM_TMP.labels
    
    return labels_arr



def onehot_enco(labelarr,df,att_name):
    og_text_arr = df[att_name].values
    #create dic to remeber loc for each value in the uniqe arr
    uni_arr = np.unique(labelarr)
    loc_dic = {}
    for i in range(len(uni_arr)):
        loc_dic[uni_arr[i]] = i
    #based on the lenth of dic,create onehot_encod
    choice = len(loc_dic)
    out = np.zeros((len(labelarr),choice+1))
    for i in range(len(labelarr)):
        #first junde if data is null
        if type(og_text_arr[i])!=str and math.isnan(og_text_arr[i]) == True:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            cur_loc = loc_dic.get(labelarr[i])
            out[i][cur_loc] = 1
            
    return out,loc_dic

def GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,attr_name,og_dic,km_num):
    labels_arr = create_onehoted_text_fixed_classification(attr_name,df,km_num)
    out_arr,loc_dic = onehot_enco(labels_arr,df,attr_name)
    #append dic to global array of dic
    #append reverse dic to overall
#     dicnew = reversedict(loc_dic)
    append_arr_of_dic_to_overall(og_dic,loc_dic,attr_name)
    p = list(np.unique(labels_arr))
    print('class_arr of nlp', p)
    return out_arr
#     return out_arr   


def Bonus_GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,attr_name,og_dic,km_num):
    labels_arr = create_onehoted_text_fixed_classification(attr_name,df,km_num)
    out_arr,loc_dic = onehot_enco(labels_arr,df,attr_name)
    #append dic to global array of dic
    #append reverse dic to overall
#     dicnew = reversedict(loc_dic)
    append_arr_of_dic_to_overall(og_dic,loc_dic,attr_name)
    p = list(np.unique(labels_arr))
    print('class_arr of nlp', p)
    return out_arr,labels_arr

# def reversedict(dic):
#     dicnew = {}
#     keyarr = list(dic.keys())
#     valarr = list(dic.values())
#     for i in range(len(keyarr)):
#         dicnew[valarr[i]] = keyarr[i]
#     return dicnew

# GENERAL WORK:
    1. MAINTAIN AN ARRAY OF DICTIONARY S.T EVERY DIC
       CONTAINS MAPPING FROM CLASSIFICAITON TO ACTUAL INDEX AFTER KMEANS
    
    2. AFTER TRAINING, WE WILL UTILIZED THE ARR_DIC TO RECOVER CANDIDATE ANSER
    
    3. TODO: REWRITE COST FUNCTION OF NERUAL NETWORK
    

# FINDING:
    AFTER combine several attributes column, num of candidate with dinct
    ans grows extremly fast.
    Need to apply Kmeans clustering again to the array of candidate before
    threw into Neural network

# CREATE A SCRIPT TO APPEND ASSIGNED ATTRIBUTE TO INPUT COLUMN

(API)INSTRUCTION:<br>
USER WHO WANTS TO CREATE PREPROCESSED DATASET ONLY NEED TO CREATE
ARRAY[i]<br>
       ARRAY[i][0] = 'name of attribute'<br>
       ARRAY[i][1] = 1 : it is a digit column<br>
       ARRAY[i][1] = 0 : it is a NLP column



In [11]:
def semi_auto_append_attri(arr_of_attribute,df,og_dic):
    for i in range(len(arr_of_attribute)):
        name = arr_of_attribute[i][0]
        isdigit = arr_of_attribute[i][1]
        if isdigit == 1:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,name,og_dic)
        else:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,name,og_dic,80)
        #append old with new
        if i == 0:
            old = attr_arr
        else:
            old = append_arr(old,attr_arr)
#             print(old.shape)
    return old
        

In [12]:
def new_semi_auto_append_attri(arr_of_attribute,df,og_dic):
    for i in range(len(arr_of_attribute)):
        name = arr_of_attribute[i][0]
        print('cur key is', name)
        isdigit = arr_of_attribute[i][1]
        if isdigit == 1:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,name,og_dic)
        elif isdigit == 0:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,name,og_dic,30)
        elif isdigit == 2:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_FIXED_CHOICE_COLUMN(df,name,og_dic)
        #append old with new
        if i == 0:
            old = attr_arr
        else:
            old = append_arr(old,attr_arr)
#             print(old.shape)
    return old

In [13]:
def fix_choice_get_distinct_arr(df,arr_str):
    arr = df[arr_str].values
    uni_arr = np.unique(arr)
    di = []
#     di_og = [] #RETURN OG SINCE di will be sorted
    div_arr = []
    for i in range(len(uni_arr)):
        if math.isnan(uni_arr[i])!=True:
            div_arr.append(uni_arr[i])
    for i in range(len(arr)):
        if math.isnan(arr[i])!= True:
            di.append(int(arr[i]))
        else:
            di.append(-1)
    return di,div_arr

def GET_ONE_HOT_ARRAY_ONLY_FIXED_CHOICE_COLUMN(df,attr_name,og_dic):
    att_arr,class_arr = fix_choice_get_distinct_arr(df,attr_name)
    print('class_arr',class_arr)
    dic_ans = create_dic_for_classification(class_arr)
    append_arr_of_dic_to_overall(og_dic,dic_ans,attr_name)
    return oneHotEncoding_return_arr(att_arr,class_arr)

# Apply Kmeans Clustering to the overal column

In [14]:
def cluster_whole_attribute_get_Y_for_Neural_NetWork(X,num_cluster):
    km = KMeans(num_cluster)
    km.fit2(X)
    
    #APPLY ONEHOT_ENCODING TO NEURAL_NETWORKAGAIN
    
    return oneHotEncoding(km.labels,km),km

# APPLY NEURAL NETWORK TO SOLVE THIS QUESTION:

REMEBER TO CONSIDER FLAG FOR EACH COLUMN

In [15]:
def process_predicted_data(pred,kmeans):
    ans = []
    for i in range(len(pred)):
        ans.append(kmeans.centers[np.argmax(pred[i])])
    newarr = np.asarray(ans)
    return newarr

def find_classification_based_on_distance(pred,dic):
    #create array with all dict keys based on insertion order
    dicarr = list(dic.keys())
    #isolate target awated_array used to classification
    for i in range(len(pred)):
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
#             print('cur dic: ',cur_dict_key)
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_lengh = len(cur_dict_arr)+1
            cur_des += cur_dict_lengh  #update current cutting position
            #restore dimension of onehotencoding for current dictionary
            one_hot_cur_dic = np.zeros((cur_dict_lengh,cur_dict_lengh))
#             print('before any processing cur loc is ', cur_loc, ' cur des is : ', cur_des)
            for q in range(0,cur_dict_lengh-1):
                one_hot_cur_dic[q][q] = 1
                one_hot_cur_dic[q][-1] = 1
            #isolate target awaited_classification array
            cur_isolated_arr = pred[i][cur_loc:cur_des]
            #compute classification with smallest distance
            arr_store_distance= []
            for h in range (len(one_hot_cur_dic)):
                arr_store_distance.append(np.linalg.norm(cur_isolated_arr-one_hot_cur_dic[h]))
            #transfer dic to arr s.t unilized argmim
            dis_np_array = np.asarray(arr_store_distance)
            index = np.argmin(dis_np_array)
#             if cur_dict_key == 'big5_10':
#                 print(index)
#                 print('cur onehot_array is ', one_hot_cur_dic[3] )
#                 print('cur loc is ', cur_loc, ' cur des is : ', cur_des)
            # change value in output array
            for k in range (cur_dict_lengh):
                pred[i][cur_loc+k] = one_hot_cur_dic[index][k]
                
            cur_loc += cur_dict_lengh #update cur_loc not used in curretn loop

def find_classification_based_on_prob(pred,dic):
    dicarr = list(dic.keys())
    for i in range(len(pred)):
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_lengh = len(cur_dict_arr)  + 1
            cur_des += cur_dict_lengh
            
            cur_isolated_arr = pred[i][cur_loc:cur_des-1]
            #find index of max value
            index = np.argmax(cur_isolated_arr)
            
            for k in range(cur_dict_lengh):
                pred[i][cur_loc+k] = 0
            
            pred[i][cur_loc + index] = 1
            cur_loc +=cur_dict_lengh
    
            
def create_reverse_dictionary(dic):
    reverse = {}
    dic_arr = list(dic.keys())
    for i in range(len(dic_arr)):
        cur_reverse_dict = {}
        cur_dic_key = dic_arr[i]
        cur_key_arr = list(dic.get(cur_dic_key).keys())
        cur_index_arr = list(dic.get(cur_dic_key).values())
        #reverse key_value pair in originaly dictionary
        for j in range(len(cur_key_arr)):
            cur_reverse_dict[cur_index_arr[j]] = cur_key_arr[j]
        #append current dic to ans dic
        reverse[cur_dic_key] = cur_reverse_dict
    return reverse

# create  function to knock out some attribute inorder to create more dataset

In [16]:
def trainingset_generator(train,dic,arr_str):
    #first find the location 
    #knock out
    #return dataset
    dicarr = list(dic.keys())
    changed_arr = np.copy(train)
    y_arr = np.copy(train)
    #outermost loop : loop through those keys that need to be cleared
    for d in range(len(arr_str)):
        cur_key = arr_str[d]
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length
            #test if we find what we want
            if cur_dict_key == cur_key:
                for i in range(len(train)):
                    for k in range(cur_dict_length):
                        changed_arr[i][cur_loc + k] = 0
                        
                break
            cur_loc +=cur_dict_length
    return changed_arr,y_arr

def append_row(old,new):
    ans = np.vstack([old,new])
    return ans


def enlarge_dataset(train,y,dic,arr_of_arr_str):
    list_of_new_x = []
    list_of_new_y = []
    new_x = np.copy(train)
    new_y = np.copy(y)
    for i in range(len(arr_of_arr_str)):
        cur_str = arr_of_arr_str[i]
        changed_arr,y_arr = trainingset_generator(train,dic,cur_str)
        #append
        new_x = append_row(new_x,changed_arr)
        new_y = append_row(new_y,y_arr)
        list_of_new_x.append(changed_arr)
        list_of_new_y.append(y_arr)
#     return new_x,new_y
    return list_of_new_x,list_of_new_y

def createlist_list_string(dic):
    ans = []
    key_arr = list(dic.keys())
    for i in range(len(key_arr)):
        tmp = []
        tmp.append(key_arr[i])
        ans.append(tmp)
        
    #degree 2
#     for i in range(len(key_arr)):
#         for j in range(i,len(key_arr)):
#             tmp = []
#             tmp.append(key_arr[i])
#             tmp.append(key_arr[j])
#             ans.append(tmp)
    return ans

def train_list_of_dataset(nn,x,y):
    nn.fit(y,y)
    for i in range(0,len(x)):
        print('current iter', i)
        nn.fit_continue(x[i],y)
        #after training compute num of correctly predicted choice for candidate
        ans = nn.predict(x[i])
        find_classification_based_on_prob(ans,nn.dic)
        count_num_of_accurately_predicted_avg(ans,X_train_new,nn.dic)
    
    return nn
    
def count_num_of_accurately_predicted_avg(x,y,dic):
    dicarr = list(dic.keys())
    
    right_predicted_count = 0
    for i in range(len(x)):
        #iterate thourhgt each choice
        tmp_right_count = 0
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dic_length = len(cur_dict_arr) + 1
            cur_des += cur_dic_length
            issame = True
            #compare x and y in range cur_loc - > cur_des
            for pointer in range(cur_loc,cur_des-1):
                if x[i][pointer] != y[i][pointer]:
                    issame = False
                    break
            
            #now check whether x[i] and y[i] are same
            if issame == True:
                tmp_right_count+=1
            cur_loc += cur_dic_length
        #finished iterate throught current candidate
        right_predicted_count+=tmp_right_count
    
    print('fianl avg right predicted count is ', right_predicted_count/len(x))
    
    
    #function used to find how well the model interpolate
#need a array contains all attribute name
#already has the function to mask every attribute
#what is output? key value pair, key: attribute name, value: avg accuracy
def evaluate(X_train_arr,Y_train_arr,og_pred,att_name_arr,hidden_neuron,iter_num):
    evaluate_dic = {}
    #to do for every attribute array utlize cross validation
    #temporary: we split first 2500 as train
    for i in range(len(X_train_arr)):
        x_cur_all = X_train_arr[i]
        y_cur_all = Y_train_arr[i]
        cur_mask_key = att_name_arr[i][0]
        #split dataset into training
        x_cur_train = x_cur_all[:2400]
        y_cur_train = y_cur_all[:2400]
        x_cur_pred = x_cur_all[2400:]
        y_cur_pred = y_cur_all[2400:]
        #create nn object
        mm = NeuralNetwork(hidden_neuron,iter_num,0.1,og_pred)
        mm.fit(x_cur_train,y_cur_train)
        #predict the answer
        predict_ans = mm.predict(x_cur_pred)
        tmp_eval_ans = count_right_predicted_num(predict_ans,y_cur_pred,og_pred,cur_mask_key)
        evaluate_dic[str(cur_mask_key)] = tmp_eval_ans
        print('cur masked name is ', cur_mask_key, ' eval value is: ', tmp_eval_ans)
        
def count_right_predicted_num(ans,y_pred,dic,att_name):
    #first find the location of masked attribute name
    #loop through dictionary list
    anscount = 0
    totallen = len(ans)
    dicarr = list(dic.keys())
    cur_loc = 0
    cur_des = 0

    for j in range(len(dicarr)):
        cur_dict_key = dicarr[j]
#         print('curloop key', cur_dict_key)
        cur_dict_arr = dic.get(cur_dict_key)
        cur_dict_length = len(cur_dict_arr) + 1
        cur_des += cur_dict_length
        #find if cur key == att_name
        if cur_dict_key == att_name:
            break
        else:
            cur_loc +=cur_dict_length
#     print('!!!!curloc ', cur_loc, ' des ', cur_des)
    #now we have location of masked attribute column, we just need to examin the answer
    #first --softmax
    find_classification_based_on_prob(ans,og_pred)
    #second -> exame
    for i in range(totallen):
        issame = True
        xcur = ans[i][cur_loc:cur_des-1]
        ycur = y_pred[i][cur_loc:cur_des-1]
        if np.array_equal(xcur,ycur):
            anscount+=1
    print('current loc and des is', cur_loc, 'and ', cur_des, ' total cound is',  anscount,'percent is', anscount/totallen)
    return anscount/totallen
    
    
    
def create_dictionary_for_every_location_of_feature(dic):
    dic_look_up = {}
    dicarr = list(dic.keys())
    cur_loc = 0
    cur_des = 0 
    for i in range(len(dicarr)):
        locarr = []
        cur_dict_key = dicarr[i]
        cur_dict_arr = dic.get(cur_dict_key)
        cur_dict_length = len(cur_dict_arr)+1
        cur_des +=cur_dict_length
        #append loccation
        locarr.append(cur_loc)
        locarr.append(cur_des)
        dic_look_up[cur_dict_key] = locarr
        cur_loc  += cur_dict_length
    return dic_look_up

# Autoencoder - Double Hidden Layer Neural Network  Implementation

In [17]:
class MULTI_NeuralNetwork:
    def __init__(self, n_h,n_h2, n_iterate=10, learning_rate=1, dic = []):
        self.n_x = None  # size of the input layer
        self.n_h = n_h  # size of the hidden layer
        self.n_h2= n_h2
        self.n_y = None # size of the output layer
        self.W1 = None
        self.W2 = None
        self.W3 = None
        self.b1 = None
        self.b2 = None
        self.b3 = None
        self.A1 = None
        self.A2 = None  # sigmoid output of the second activation
        self.A3 = None
        self.learning_rate = learning_rate
        self.n_iterate = n_iterate
#         self.dic = dic
        self.dic = dic
    
    def initialize_parameters(self):
        self.W1 = np.random.randn(self.n_h, self.n_x) * 0.01
        self.b1 = np.zeros((self.n_h, 1))
        
        self.W2 = np.random.randn(self.n_h2, self.n_h) * 0.01
        self.b2 = np.zeros((self.n_h2, 1))
        
        self.W3 = np.random.randn(self.n_y, self.n_h2) * 0.01
        self.b3 = np.zeros((self.n_y, 1))
    
    
    def MSE(self,Y):
        cost = 0
        for i in range(len(Y)):
            for j in range(len(Y[0])):
                cost+=(Y[i][j]-self.A2[i][j])**2
        cost = cost/len(Y)
        return cost
        
    def relu(self, z):
        return z * (z > 0)
    
#     def softmax(self,z):
#         exps = np.exp(z-np.max(z,axis= 1,keepdims = True))
#         return exps/np.sum(exps,axis = 1, keepdims = True)
    
    

#     def softmax(self,A):  
#         expA = np.exp(A)
#         return expA / expA.sum(axis=1, keepdims=True)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derv(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    
    
    def cro_entro(self,Y):
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean()
        return cost
    
    def compute_cost(self, Y):# cross entrophy
        #cost = np.linalg.norm(self.A2 - Y)
        cost = - (Y * np.log(self.A3) + (1-Y) * np.log(1-self.A3)).mean()
#         print('shape cost is: ' ,cost.shape)
        return np.squeeze(cost) 

    def forward_propagation(self, X):
        self.A1 = self.relu(self.W1 @ X + self.b1)
        #dropout function for output in hidden layer
        u1 = np.random.binomial(1,0.3,size = self.A1.shape)
        self.A1 *= u1

        self.A2 = self.relu(self.W2 @ self.A1 + self.b2)
        #dropout function for hidden layer two
        u2 = np.random.binomial(1,0.3,size = self.A2.shape)
        self.A2 *= u2
        
        self.A3 = self.sigmoid(self.W3 @self.A2 + self.b3)
        

        
#         u2 = np.random.binomial(1,0.5,size = self.A2.shape)
#         self.A2 *= u2
#         self.A2 = self.soft_max_For_multi_attribute(self.A2,self.dic)

        
    
    def backward_propagation(self, X, Y):
        m = X.shape[1]  #num of input candidate

#         dZ2 = self.A2 - Y
# #         print('cur shape of dz2 :',dZ2.shape)
# #         print('cur cost of cros entro', c)
#         dW2 = dZ2 @ self.A1.T / m
#         db2 = np.sum(dZ2, axis=1, keepdims=True) / m
#         dZ1 = self.W2.T @ dZ2 * (self.A1 > 0)
#         dW1 = dZ1 @ X.T / m
#         db1 = np.sum(dZ1, axis=1, keepdims=True) / m

#         self.W1 -= self.learning_rate * dW1
#         self.b1 -= self.learning_rate * db1
#         self.W2 -= self.learning_rate * dW2
#         self.b2 -= self.learning_rate * db2

# <------------------------------------------------------>
#           used for two sigmoid hidden layer
#         dZ3 = (self.A3 - Y)/m
#         dW3 = np.dot(dZ3 , self.A2.T) 
#         db3 = np.sum(dZ3,axis = 1, keepdims = True)
#         dZ2 = np.dot(self.W3.T,  dZ3) * self.sigmoid_derv(self.A2)
#         dW2 = dZ2 @self.A1.T
#         db2 = np.sum(dZ2,axis = 1, keepdims = True)
#         dZ1 = np.dot(self.W2.T , dZ2) * self.sigmoid_derv(self.A1)
#         dW1 = dZ1@ X.T
#         db1 = np.sum(dZ1,axis = 1, keepdims = True)
# <------------------------------------------------------->


        dZ3 = (self.A3 - Y)/m
        dW3 = np.dot(dZ3 , self.A2.T) 
        db3 = np.sum(dZ3,axis = 1, keepdims = True)
        dZ2 = np.dot(self.W3.T,  dZ3) * (self.A2 > 0)
        dW2 = dZ2 @self.A1.T
        db2 = np.sum(dZ2,axis = 1, keepdims = True)
        dZ1 = np.dot(self.W2.T , dZ2) * (self.A1 > 0)
        dW1 = dZ1@ X.T
        db1 = np.sum(dZ1,axis = 1, keepdims = True)
        
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W3 -= self.learning_rate * dW3
        self.b3 -= self.learning_rate * db3
    
    def fit(self, X, Y):
        X, Y = X.T, Y.T
    
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
        self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                  self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))

    def fit_continue(self,X,Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
#         if self.W1 is None:
#             self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                 self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))
                
    def predict(self, X):
        X = X.T
        A1 = self.sigmoid(self.W1 @ X + self.b1)
        A2 = self.sigmoid(self.W2 @ A1 + self.b2)
        A3 = self.sigmoid(self.W3 @ A2 + self.b3)
        
        return A3.T
    
    def predict_one_layer(self,X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)
        return A1.T
    
    def soft_max_For_multi_attribute(self,arr,dic):
        #create array with all dict keys based on insertion order
        tmp_arr = np.copy(arr)
        dicarr = list(dic.keys())
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length    
            #find the targeted array need to perform softmax
            #create a dic to contain the location of those onehotencoding attribute
            loc_dic = []
            for i in range(cur_dict_length-1):
                loc_dic.append(cur_loc+i)
            #find targed array
            target = tmp_arr[:,loc_dic]
            #get softmaxed arr
            arr_soft_max = self.api_softmax(target)
            #reassig value back to orignal arr
            tmp_arr[:,loc_dic] = arr_soft_max
            #update cur_loc
            cur_loc +=cur_dict_length
            
        return tmp_arr

    def api_softmax(self,z):
        exps = np.exp(z)
        return exps/np.sum(exps,axis = 1, keepdims = True)

In [1]:
#function to find mutual dependency feature
def findvalue_feature(final):
    #get feature keys
    fianldic = {}
    keyarr = list(final.keys())
    #loop each key
    for i in range(len(keyarr)):
        curlist = final.get(keyarr[i])
        #1.find the smallest value
        smallest = 1000
        #loop through
        for j in range(len(curlist)):
            curval = list(curlist[j].values())[0]
            if curval < smallest:
                smallest = curval
        # already find the smallest val
        #2.find the index of smallest value
        indexlist = []
        for p in range(len(curlist)):
            curval = list(curlist[p].values())[0]
            if curval == smallest:
                indexlist.append(p)
        #3.create an list to append name of these value
        featurenamelist = []
        for l in range(len(indexlist)):
            pointer = indexlist[l]
            #get key
            curkey = list(curlist[pointer].keys())[0]
            #append to list
            featurenamelist.append(curkey)
        #save result to an dict
        print('cur key is', keyarr[i])
        fianldic [keyarr[i]] = featurenamelist
    return fianldic

In [18]:
#write function to complete quesiton: Analyze the Data
#step1 : use the answer from evalue function - > those feature that has high dependencies
#first find those masked vector based on the name of features with high depen
#step2 : try also mask every other feature to which additional maskede feature lead to smallest accurayc
#of interpolating
def Analyzethedata(x,y,dic,arr_of_feature_high_mutual_dependencies,masked_arr,dic_location,hidden_neuron_num,iter_num,finalans):
    for i in range(len(arr_of_feature_high_mutual_dependencies)):
        for j in range(len(masked_arr)):
            if arr_of_feature_high_mutual_dependencies[i] == masked_arr[j][0]:
                cur_key_ans_arr = []
                #now current index j could be used to find the masked array
                x_feature_masked_high_muldependencies = x[j]
                y_feature_masked_high_muldependencies = y[j]
                cur_masked_key = arr_of_feature_high_mutual_dependencies[i]
                #step 2 loop throught every other key
                #first aplit dataset into training and preding
                for k in range(len(masked_arr)):
                    if masked_arr[k][0]!= cur_masked_key:
                        print('current masked key is', cur_masked_key, ' and other key is', masked_arr[k][0])
                        #masked other feature  -> first find the location of other feaure in one hot encoded array
                        x_cur_tmp_feature_masked_high_muldenpendencies = np.copy(x_feature_masked_high_muldependencies)
                        y_cur_tmp_feature_masked_high_muldenpendencies = np.copy(y_feature_masked_high_muldependencies)
                        cur_key = masked_arr[k][0]
                        #find location
                        cur_loc = dic_location.get(cur_key)[0]
                        cur_des = dic_location.get(cur_key)[1]
                        print('cur masked loc', cur_loc, 'cur des ', cur_des)
                        #mask
                        for pointer in range(len(x_cur_tmp_feature_masked_high_muldenpendencies)):
                            for pointer2 in range(cur_loc,cur_des):
                                x_cur_tmp_feature_masked_high_muldenpendencies[pointer][pointer2] = 0
                        #mask process complete
                        
                        #split dataset into training and predicting
                        x_cur_train = x_cur_tmp_feature_masked_high_muldenpendencies[:2000]
                        y_cur_train = y_cur_tmp_feature_masked_high_muldenpendencies[:2000]
                        x_cur_pred = x_cur_tmp_feature_masked_high_muldenpendencies[2000:]
                        y_cur_pred = y_cur_tmp_feature_masked_high_muldenpendencies[2000:]
                        nn = NeuralNetwork(hidden_neuron_num,iter_num,0.1,dic)
                        nn.fit(x_cur_train,y_cur_train)
                        #predict the asnwer
                        predict_ans = nn.predict(x_cur_pred)
                        totallen = len(predict_ans)
                        tmp_eval_ans = count_right_predicted_num(predict_ans,y_cur_pred,dic,cur_masked_key)
                        print('curmasked key is', cur_masked_key,'cur_iter_key is', cur_key,'percent is', tmp_eval_ans)
                        cur_dic = {}
                        cur_dic[cur_key] = tmp_eval_ans
                        cur_key_ans_arr.append(cur_dic)
                finalans[cur_masked_key] = cur_key_ans_arr
    return finalans

In [19]:
def createfeaturearr():
    arr = [] 
    arr.append(['age',1])
    arr.append(['mood_01',2]) #Today I generally feel
    arr.append(['mood_02',2])
    arr.append(['major',0])
    arr.append(['big5_01',2]) #I see myself as: Extraverted, enthusiastic.
    arr.append(['big5_02',2]) #I see myself as: Critical, quarrelsome.
    arr.append(['big5_03',2]) #I see myself as: Dependable, self-disciplined.
    arr.append(['big5_04',2]) #I see myself as: Anxious, easily upset.
    arr.append(['big5_05',2]) #I see myself as: Open to new experiences, complex.
    arr.append(['big5_06',2]) #I see myself as: Reserved, quiet.
    arr.append(['big5_07',2]) #I see myself as: Sympathetic, warm.
    arr.append(['big5_08',2]) #I see myself as: Disorganized, careless.
    arr.append(['big5_09',2]) #I see myself as: Calm, emotionally stable.
    arr.append(['big5_10',2]) #I see myself as: Conventional, uncreative.
    arr.append(['highpower',0])
    arr.append(['intrinsic_01',2])
    arr.append(['intrinsic_02',2])
    arr.append(['intrinsic_03',2])
    arr.append(['intrinsic_04',2])
    arr.append(['intrinsic_05',2])
    arr.append(['intrinsic_06',2])
    arr.append(['intrinsic_07',2])
    arr.append(['intrinsic_08',2])
    arr.append(['intrinsic_09',2])
    arr.append(['intrinsic_10',2])
    arr.append(['intrinsic_11',2])
    arr.append(['intrinsic_12',2])
    arr.append(['intrinsic_13',2])
    arr.append(['intrinsic_14',2])
    arr.append(['intrinsic_15',2])
    arr.append(['lowpower',0])
    arr.append(['mcfiller1',2])
    arr.append(['mcfiller2',2])
    arr.append(['mcfiller3',2])
    arr.append(['mcmost1',2])
    arr.append(['mcmost2',2])
    arr.append(['mcmost3',2])
    arr.append(['mcmost4',2])
    arr.append(['mcmost5',2])
    arr.append(['mcsome1',2])
    arr.append(['mcsome2',2])
    arr.append(['mcsome3',2])
    arr.append(['mcsome4',2])
    arr.append(['mcsome5',2])
    arr.append(['mcdv1',2])
    arr.append(['mcdv2',2])
    arr.append(['pate_01',2])
    arr.append(['pate_02',2])
    arr.append(['pate_03',2])
    arr.append(['pate_04',2])
    arr.append(['pate_05',2])
    arr.append(['stress_01',2])
    arr.append(['stress_02',2])
    arr.append(['stress_03',2])
    arr.append(['stress_04',2])
    arr.append(['nfc_01',2])
    arr.append(['nfc_02',2])
    arr.append(['nfc_03',2])
    arr.append(['nfc_04',2])
    arr.append(['nfc_05',2])
    arr.append(['nfc_06',2])
    arr.append(['tempest1',0])
    arr.append(['tempest2',2])
    arr.append(['tempest3',2])
    return arr

# After all compliting, Run code below to do question 'Evaluate your model Validation'

In [25]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr.append(['age',1])
arr.append(['mood_01',2]) #Today I generally feel
arr.append(['mood_02',2])
arr.append(['major',0])
arr.append(['big5_01',2]) #I see myself as: Extraverted, enthusiastic.
arr.append(['big5_02',2]) #I see myself as: Critical, quarrelsome.
arr.append(['big5_03',2]) #I see myself as: Dependable, self-disciplined.
arr.append(['big5_04',2]) #I see myself as: Anxious, easily upset.
arr.append(['big5_05',2]) #I see myself as: Open to new experiences, complex.
arr.append(['big5_06',2]) #I see myself as: Reserved, quiet.
arr.append(['big5_07',2]) #I see myself as: Sympathetic, warm.
arr.append(['big5_08',2]) #I see myself as: Disorganized, careless.
arr.append(['big5_09',2]) #I see myself as: Calm, emotionally stable.
arr.append(['big5_10',2]) #I see myself as: Conventional, uncreative.
arr.append(['highpower',0])
arr.append(['intrinsic_01',2])
arr.append(['intrinsic_02',2])
arr.append(['intrinsic_03',2])
arr.append(['intrinsic_04',2])
arr.append(['intrinsic_05',2])
arr.append(['intrinsic_06',2])
arr.append(['intrinsic_07',2])
arr.append(['intrinsic_08',2])
arr.append(['intrinsic_09',2])
arr.append(['intrinsic_10',2])
arr.append(['intrinsic_11',2])
arr.append(['intrinsic_12',2])
arr.append(['intrinsic_13',2])
arr.append(['intrinsic_14',2])
arr.append(['intrinsic_15',2])
arr.append(['lowpower',0])
arr.append(['mcfiller1',2])
arr.append(['mcfiller2',2])
arr.append(['mcfiller3',2])
arr.append(['mcmost1',2])
arr.append(['mcmost2',2])
arr.append(['mcmost3',2])
arr.append(['mcmost4',2])
arr.append(['mcmost5',2])
arr.append(['mcsome1',2])
arr.append(['mcsome2',2])
arr.append(['mcsome3',2])
arr.append(['mcsome4',2])
arr.append(['mcsome5',2])
arr.append(['mcdv1',2])
arr.append(['mcdv2',2])
arr.append(['pate_01',2])
arr.append(['pate_02',2])
arr.append(['pate_03',2])
arr.append(['pate_04',2])
arr.append(['pate_05',2])
arr.append(['stress_01',2])
arr.append(['stress_02',2])
arr.append(['stress_03',2])
arr.append(['stress_04',2])
arr.append(['nfc_01',2])
arr.append(['nfc_02',2])
arr.append(['nfc_03',2])
arr.append(['nfc_04',2])
arr.append(['nfc_05',2])
arr.append(['nfc_06',2])

In [20]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)

cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

In [21]:
X_train_new = X_train[:2900]
#slice nparr
x_real_train = X_train[:2900]
x_pred = X_train[2900:]

In [22]:
test = createlist_list_string(og_pred)
x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

# Output evaluation result

In [40]:
di = evaluate(x,y,og_pred,test,2000,200)

n_x is 334 n_y is  334
Cost after iteration 0: 0.693137
Cost after iteration 10: 0.608048
Cost after iteration 20: 0.366226
Cost after iteration 30: 0.321158
Cost after iteration 40: 0.288153
Cost after iteration 50: 0.275451
Cost after iteration 60: 0.262345
Cost after iteration 70: 0.256449
Cost after iteration 80: 0.247787
Cost after iteration 90: 0.243578
Cost after iteration 100: 0.237524
Cost after iteration 110: 0.232126
Cost after iteration 120: 0.227163
Cost after iteration 130: 0.223249
Cost after iteration 140: 0.218627
Cost after iteration 150: 0.212329
Cost after iteration 160: 0.210667
Cost after iteration 170: 0.204694
Cost after iteration 180: 0.199388
Cost after iteration 190: 0.193867
current loc and des is 0 and  4  total cound is 185 percent is 0.37
cur masked name is  age  eval value is:  0.37
n_x is 334 n_y is  334
Cost after iteration 0: 0.693338
Cost after iteration 10: 0.608776
Cost after iteration 20: 0.366608
Cost after iteration 30: 0.321556
Cost after itera

current loc and des is 68 and  76  total cound is 167 percent is 0.334
cur masked name is  big5_07  eval value is:  0.334
n_x is 334 n_y is  334
Cost after iteration 0: 0.693349
Cost after iteration 10: 0.602155
Cost after iteration 20: 0.368933
Cost after iteration 30: 0.321973
Cost after iteration 40: 0.288587
Cost after iteration 50: 0.277427
Cost after iteration 60: 0.260672
Cost after iteration 70: 0.254386
Cost after iteration 80: 0.246495
Cost after iteration 90: 0.242232
Cost after iteration 100: 0.237617
Cost after iteration 110: 0.233832
Cost after iteration 120: 0.227371
Cost after iteration 130: 0.220197
Cost after iteration 140: 0.213409
Cost after iteration 150: 0.215503
Cost after iteration 160: 0.205877
Cost after iteration 170: 0.205322
Cost after iteration 180: 0.200915
Cost after iteration 190: 0.189226
current loc and des is 76 and  84  total cound is 130 percent is 0.26
cur masked name is  big5_08  eval value is:  0.26
n_x is 334 n_y is  334
Cost after iteration 0:

Cost after iteration 150: 0.211830
Cost after iteration 160: 0.207758
Cost after iteration 170: 0.206549
Cost after iteration 180: 0.199966
Cost after iteration 190: 0.201031
current loc and des is 133 and  139  total cound is 222 percent is 0.444
cur masked name is  intrinsic_07  eval value is:  0.444
n_x is 334 n_y is  334
Cost after iteration 0: 0.692844
Cost after iteration 10: 0.608390
Cost after iteration 20: 0.365803
Cost after iteration 30: 0.323466
Cost after iteration 40: 0.290505
Cost after iteration 50: 0.283459
Cost after iteration 60: 0.264228
Cost after iteration 70: 0.257194
Cost after iteration 80: 0.246894
Cost after iteration 90: 0.240349
Cost after iteration 100: 0.236990
Cost after iteration 110: 0.230340
Cost after iteration 120: 0.228695
Cost after iteration 130: 0.224564
Cost after iteration 140: 0.218090
Cost after iteration 150: 0.214751
Cost after iteration 160: 0.210779
Cost after iteration 170: 0.202022
Cost after iteration 180: 0.199342
Cost after iteratio

Cost after iteration 90: 0.253943
Cost after iteration 100: 0.235575
Cost after iteration 110: 0.232020
Cost after iteration 120: 0.227651
Cost after iteration 130: 0.222736
Cost after iteration 140: 0.216552
Cost after iteration 150: 0.211640
Cost after iteration 160: 0.204418
Cost after iteration 170: 0.203054
Cost after iteration 180: 0.197679
Cost after iteration 190: 0.191775
current loc and des is 187 and  193  total cound is 265 percent is 0.53
cur masked name is  mcfiller2  eval value is:  0.53
n_x is 334 n_y is  334
Cost after iteration 0: 0.693167
Cost after iteration 10: 0.604700
Cost after iteration 20: 0.363842
Cost after iteration 30: 0.320261
Cost after iteration 40: 0.287123
Cost after iteration 50: 0.280987
Cost after iteration 60: 0.261228
Cost after iteration 70: 0.251362
Cost after iteration 80: 0.244916
Cost after iteration 90: 0.242042
Cost after iteration 100: 0.234255
Cost after iteration 110: 0.232791
Cost after iteration 120: 0.225783
Cost after iteration 130:

Cost after iteration 50: 0.279033
Cost after iteration 60: 0.261217
Cost after iteration 70: 0.253278
Cost after iteration 80: 0.247042
Cost after iteration 90: 0.243837
Cost after iteration 100: 0.236794
Cost after iteration 110: 0.237149
Cost after iteration 120: 0.227932
Cost after iteration 130: 0.222578
Cost after iteration 140: 0.217393
Cost after iteration 150: 0.218502
Cost after iteration 160: 0.209406
Cost after iteration 170: 0.207284
Cost after iteration 180: 0.203524
Cost after iteration 190: 0.196796
current loc and des is 222 and  225  total cound is 140 percent is 0.28
cur masked name is  mcsome4  eval value is:  0.28
n_x is 334 n_y is  334
Cost after iteration 0: 0.692947
Cost after iteration 10: 0.613454
Cost after iteration 20: 0.370950
Cost after iteration 30: 0.330205
Cost after iteration 40: 0.289781
Cost after iteration 50: 0.268393
Cost after iteration 60: 0.259171
Cost after iteration 70: 0.253818
Cost after iteration 80: 0.247480
Cost after iteration 90: 0.246

Cost after iteration 10: 0.602379
Cost after iteration 20: 0.366447
Cost after iteration 30: 0.318851
Cost after iteration 40: 0.285729
Cost after iteration 50: 0.281740
Cost after iteration 60: 0.260108
Cost after iteration 70: 0.252315
Cost after iteration 80: 0.244432
Cost after iteration 90: 0.243589
Cost after iteration 100: 0.235165
Cost after iteration 110: 0.229579
Cost after iteration 120: 0.224059
Cost after iteration 130: 0.225406
Cost after iteration 140: 0.220312
Cost after iteration 150: 0.217616
Cost after iteration 160: 0.204355
Cost after iteration 170: 0.208463
Cost after iteration 180: 0.197274
Cost after iteration 190: 0.192505
current loc and des is 280 and  286  total cound is 205 percent is 0.41
cur masked name is  stress_02  eval value is:  0.41
n_x is 334 n_y is  334
Cost after iteration 0: 0.693432
Cost after iteration 10: 0.604937
Cost after iteration 20: 0.367456
Cost after iteration 30: 0.321406
Cost after iteration 40: 0.287767
Cost after iteration 50: 0.2

# Model Evaluation proces done

# 

# 

# 

# Answer Question in ' Analyze the Data'
# for Computation consuming reason,
# only run Neural network 70 epochs, our Model did much better job with higher epoches

In [20]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr = createfeaturearr()

In [21]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)
X_train_new = X_train[:2500]
#slice nparr
x_real_train = X_train[:2500]
x_pred = X_train[2500:]

test = createlist_list_string(og_pred)
# x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

In [22]:
test = createlist_list_string(og_pred)
x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

# a is a list of feature that is highly dependent with other feature found the previous question

In [23]:
a = ['mood_02,','intrinsic_01','intrinsic_02','intrinsic_03','intrinsic_04'
    ,'intrinsic_08','intrinsic_10','intrinsic_13','intrinsic_15','mcfiller1'
     ,'mcfiller2','mcfiller3','mcdv1','pate_03','pate_04','stress_02','stress_03'
     ,'nfc_02','nfc_04','nfc_05','tempest1','tempest2','tempest3']

In [24]:
finaldic = {} 
final = Analyzethedata(x,y,og_pred,a,test,dic_look,2000,70,finaldic)

current masked key is intrinsic_01  and other key is age
cur masked loc 0 cur des  4
n_x is 405 n_y is  405
Cost after iteration 0: 0.692936
Cost after iteration 10: 0.485090
Cost after iteration 20: 0.587432
Cost after iteration 30: 0.291109
Cost after iteration 40: 0.260094
Cost after iteration 50: 0.245046
Cost after iteration 60: 0.235455
current loc and des is 131 and  136  total cound is 232 percent is 0.464
curmasked key is intrinsic_01 cur_iter_key is age percent is 0.464
current masked key is intrinsic_01  and other key is mood_01
cur masked loc 4 cur des  12
n_x is 405 n_y is  405
Cost after iteration 0: 0.693199
Cost after iteration 10: 0.451961
Cost after iteration 20: 0.520579
Cost after iteration 30: 0.344668
Cost after iteration 40: 0.276508
Cost after iteration 50: 0.252624
Cost after iteration 60: 0.245292
current loc and des is 131 and  136  total cound is 232 percent is 0.464
curmasked key is intrinsic_01 cur_iter_key is mood_01 percent is 0.464
current masked key is

Cost after iteration 50: 0.246669
Cost after iteration 60: 0.238537
current loc and des is 131 and  136  total cound is 267 percent is 0.534
curmasked key is intrinsic_01 cur_iter_key is intrinsic_03 percent is 0.534
current masked key is intrinsic_01  and other key is intrinsic_04
cur masked loc 147 cur des  153
n_x is 405 n_y is  405
Cost after iteration 0: 0.692919
Cost after iteration 10: 0.549414
Cost after iteration 20: 0.403751
Cost after iteration 30: 0.294665
Cost after iteration 40: 0.258180
Cost after iteration 50: 0.245457
Cost after iteration 60: 0.241820
current loc and des is 131 and  136  total cound is 232 percent is 0.464
curmasked key is intrinsic_01 cur_iter_key is intrinsic_04 percent is 0.464
current masked key is intrinsic_01  and other key is intrinsic_05
cur masked loc 153 cur des  159
n_x is 405 n_y is  405
Cost after iteration 0: 0.693323
Cost after iteration 10: 0.461829
Cost after iteration 20: 0.392045
Cost after iteration 30: 0.296139
Cost after iteration

Cost after iteration 0: 0.692954
Cost after iteration 10: 0.488449
Cost after iteration 20: 0.533394
Cost after iteration 30: 0.463640
Cost after iteration 40: 0.283090
Cost after iteration 50: 0.255465
Cost after iteration 60: 0.246260
current loc and des is 131 and  136  total cound is 244 percent is 0.488
curmasked key is intrinsic_01 cur_iter_key is mcmost1 percent is 0.488
current masked key is intrinsic_01  and other key is mcmost2
cur masked loc 241 cur des  244
n_x is 405 n_y is  405
Cost after iteration 0: 0.693014
Cost after iteration 10: 0.445763
Cost after iteration 20: 0.394446
Cost after iteration 30: 0.304234
Cost after iteration 40: 0.260200
Cost after iteration 50: 0.246899
Cost after iteration 60: 0.243970
current loc and des is 131 and  136  total cound is 263 percent is 0.526
curmasked key is intrinsic_01 cur_iter_key is mcmost2 percent is 0.526
current masked key is intrinsic_01  and other key is mcmost3
cur masked loc 244 cur des  247
n_x is 405 n_y is  405
Cost a

Cost after iteration 0: 0.693057
Cost after iteration 10: 0.494503
Cost after iteration 20: 0.418578
Cost after iteration 30: 0.360332
Cost after iteration 40: 0.268841
Cost after iteration 50: 0.249395
Cost after iteration 60: 0.242116
current loc and des is 131 and  136  total cound is 232 percent is 0.464
curmasked key is intrinsic_01 cur_iter_key is stress_01 percent is 0.464
current masked key is intrinsic_01  and other key is stress_02
cur masked loc 320 cur des  326
n_x is 405 n_y is  405
Cost after iteration 0: 0.693186
Cost after iteration 10: 0.423562
Cost after iteration 20: 0.342701
Cost after iteration 30: 0.280865
Cost after iteration 40: 0.256053
Cost after iteration 50: 0.243419
Cost after iteration 60: 0.235667
current loc and des is 131 and  136  total cound is 232 percent is 0.464
curmasked key is intrinsic_01 cur_iter_key is stress_02 percent is 0.464
current masked key is intrinsic_01  and other key is stress_03
cur masked loc 326 cur des  332
n_x is 405 n_y is  40

Cost after iteration 0: 0.693020
Cost after iteration 10: 0.459201
Cost after iteration 20: 0.336361
Cost after iteration 30: 0.283410
Cost after iteration 40: 0.253537
Cost after iteration 50: 0.242932
Cost after iteration 60: 0.234640
current loc and des is 136 and  141  total cound is 262 percent is 0.524
curmasked key is intrinsic_02 cur_iter_key is big5_01 percent is 0.524
current masked key is intrinsic_02  and other key is big5_02
cur masked loc 42 cur des  50
n_x is 405 n_y is  405
Cost after iteration 0: 0.693423
Cost after iteration 10: 0.481934
Cost after iteration 20: 0.370652
Cost after iteration 30: 0.287370
Cost after iteration 40: 0.259418
Cost after iteration 50: 0.246611
Cost after iteration 60: 0.237474
current loc and des is 136 and  141  total cound is 215 percent is 0.43
curmasked key is intrinsic_02 cur_iter_key is big5_02 percent is 0.43
current masked key is intrinsic_02  and other key is big5_03
cur masked loc 50 cur des  58
n_x is 405 n_y is  405
Cost after i

...

Cost after iteration 0: 0.693352
Cost after iteration 10: 0.459230
Cost after iteration 20: 0.632533
Cost after iteration 30: 0.307493
Cost after iteration 40: 0.271680
Cost after iteration 50: 0.252732
Cost after iteration 60: 0.241626
current loc and des is 392 and  400  total cound is 166 percent is 0.332
curmasked key is tempest2 cur_iter_key is stress_03 percent is 0.332
current masked key is tempest2  and other key is stress_04
cur masked loc 332 cur des  338
n_x is 405 n_y is  405
Cost after iteration 0: 0.692962
Cost after iteration 10: 0.543504
Cost after iteration 20: 0.535201
Cost after iteration 30: 0.299714
Cost after iteration 40: 0.259493
Cost after iteration 50: 0.247301
Cost after iteration 60: 0.243987
current loc and des is 392 and  400  total cound is 187 percent is 0.374
curmasked key is tempest2 cur_iter_key is stress_04 percent is 0.374
current masked key is tempest2  and other key is nfc_01
cur masked loc 338 cur des  344
n_x is 405 n_y is  405
Cost after iterat

Cost after iteration 0: 0.693014
Cost after iteration 10: 0.489435
Cost after iteration 20: 0.382980
Cost after iteration 30: 0.293797
Cost after iteration 40: 0.254957
Cost after iteration 50: 0.244578
Cost after iteration 60: 0.241796
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is big5_04 percent is 0.318
current masked key is tempest3  and other key is big5_05
cur masked loc 66 cur des  74
n_x is 405 n_y is  405
Cost after iteration 0: 0.693173
Cost after iteration 10: 0.540866
Cost after iteration 20: 0.370844
Cost after iteration 30: 0.316452
Cost after iteration 40: 0.257271
Cost after iteration 50: 0.244836
Cost after iteration 60: 0.244048
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is big5_05 percent is 0.318
current masked key is tempest3  and other key is big5_06
cur masked loc 74 cur des  82
n_x is 405 n_y is  405
Cost after iteration 0: 0.

Cost after iteration 0: 0.693174
Cost after iteration 10: 0.441472
Cost after iteration 20: 0.463770
Cost after iteration 30: 0.327714
Cost after iteration 40: 0.262348
Cost after iteration 50: 0.247965
Cost after iteration 60: 0.242301
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is intrinsic_10 percent is 0.318
current masked key is tempest3  and other key is intrinsic_11
cur masked loc 186 cur des  192
n_x is 405 n_y is  405
Cost after iteration 0: 0.692943
Cost after iteration 10: 0.446059
Cost after iteration 20: 0.336480
Cost after iteration 30: 0.287217
Cost after iteration 40: 0.255726
Cost after iteration 50: 0.246768
Cost after iteration 60: 0.235201
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is intrinsic_11 percent is 0.318
current masked key is tempest3  and other key is intrinsic_12
cur masked loc 192 cur des  197
n_x is 405 n_y is  405
Co

Cost after iteration 0: 0.693363
Cost after iteration 10: 0.471417
Cost after iteration 20: 1.132909
Cost after iteration 30: 0.311412
Cost after iteration 40: 0.256826
Cost after iteration 50: 0.244763
Cost after iteration 60: 0.239385
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is mcsome3 percent is 0.318
current masked key is tempest3  and other key is mcsome4
cur masked loc 262 cur des  265
n_x is 405 n_y is  405
Cost after iteration 0: 0.692985
Cost after iteration 10: 0.438200
Cost after iteration 20: 0.495655
Cost after iteration 30: 0.314600
Cost after iteration 40: 0.266292
Cost after iteration 50: 0.248689
Cost after iteration 60: 0.242936
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is mcsome4 percent is 0.318
current masked key is tempest3  and other key is mcsome5
cur masked loc 265 cur des  268
n_x is 405 n_y is  405
Cost after iteration 0

Cost after iteration 0: 0.693073
Cost after iteration 10: 0.528347
Cost after iteration 20: 0.359142
Cost after iteration 30: 0.298771
Cost after iteration 40: 0.260791
Cost after iteration 50: 0.247028
Cost after iteration 60: 0.240643
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is nfc_04 percent is 0.318
current masked key is tempest3  and other key is nfc_05
cur masked loc 362 cur des  368
n_x is 405 n_y is  405
Cost after iteration 0: 0.692987
Cost after iteration 10: 0.499626
Cost after iteration 20: 0.341301
Cost after iteration 30: 0.285227
Cost after iteration 40: 0.255144
Cost after iteration 50: 0.241814
Cost after iteration 60: 0.235155
current loc and des is 400 and  405  total cound is 159 percent is 0.318
curmasked key is tempest3 cur_iter_key is nfc_05 percent is 0.318
current masked key is tempest3  and other key is nfc_06
cur masked loc 368 cur des  374
n_x is 405 n_y is  405
Cost after iteration 0: 0.

In [26]:
finaldictans = findvalue_feature(final)

cur key is intrinsic_01
cur key is intrinsic_02
cur key is intrinsic_03
cur key is intrinsic_04
cur key is intrinsic_08
cur key is intrinsic_10
cur key is intrinsic_13
cur key is intrinsic_15
cur key is mcfiller1
cur key is mcfiller2
cur key is mcfiller3
cur key is mcdv1
cur key is pate_03
cur key is pate_04
cur key is stress_02
cur key is stress_03
cur key is nfc_02
cur key is nfc_04
cur key is nfc_05
cur key is tempest1
cur key is tempest2
cur key is tempest3


In [27]:
finaldictans

{'intrinsic_01': ['age',
  'mood_01',
  'big5_01',
  'big5_07',
  'big5_10',
  'highpower',
  'intrinsic_04',
  'intrinsic_08',
  'mcfiller1',
  'mcfiller2',
  'mcmost3',
  'mcmost5',
  'mcsome4',
  'mcsome5',
  'mcdv1',
  'pate_03',
  'stress_01',
  'stress_02',
  'stress_03',
  'nfc_04',
  'tempest1',
  'tempest2',
  'tempest3'],
 'intrinsic_02': ['lowpower'],
 'intrinsic_03': ['age'],
 'intrinsic_04': ['mood_02',
  'big5_02',
  'intrinsic_06',
  'mcfiller2',
  'mcfiller3',
  'mcsome3'],
 'intrinsic_08': ['intrinsic_01',
  'intrinsic_03',
  'intrinsic_13',
  'intrinsic_15',
  'mcmost4',
  'nfc_01'],
 'intrinsic_10': ['pate_01'],
 'intrinsic_13': ['big5_08', 'mcmost2'],
 'intrinsic_15': ['intrinsic_14'],
 'mcfiller1': ['pate_02'],
 'mcfiller2': ['age',
  'mood_01',
  'mood_02',
  'major',
  'big5_01',
  'big5_02',
  'big5_03',
  'big5_04',
  'big5_05',
  'big5_06',
  'big5_07',
  'big5_08',
  'big5_09',
  'big5_10',
  'highpower',
  'intrinsic_04',
  'intrinsic_06',
  'intrinsic_07',


# Save result of feature dependency dictionary to csv

# We convert this csv file to edge list in file 'Convert dependencies graph to edge list .ipynb'

#  Use 'gephi ' to visualize result by using edge list

In [61]:
fi = pd.DataFrame(finaldictans)

In [62]:
fi.to_csv("./Analyzethedata.csv",index=False,header=True)

# 

# Answering question in ' Generate Data' now

In [66]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr = createfeaturearr()

In [67]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)
X_train_new = X_train[:2500]
#slice nparr
x_real_train = X_train[:2500]
x_pred = X_train[2500:]

test = createlist_list_string(og_pred)
# x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)


cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

In [2]:
# genertate realistic dataset 
# mask part of feature  (done)
# split dataset into training and testing (done)
# find the new dataset (done)
# recover dataset to human readiable answer from one hot encoding to human answer

In [27]:
nn = NeuralNetwork(2000,1000,0.1)

In [28]:
nn.fit(X_train_new,X_train_new)

n_x is 374 n_y is  374
Cost after iteration 0: 0.693280
Cost after iteration 10: 0.448408
Cost after iteration 20: 0.364520
Cost after iteration 30: 0.308424
Cost after iteration 40: 0.274802
Cost after iteration 50: 0.260301
Cost after iteration 60: 0.248980
Cost after iteration 70: 0.239949
Cost after iteration 80: 0.235451
Cost after iteration 90: 0.233066
Cost after iteration 100: 0.226620
Cost after iteration 110: 0.219310
Cost after iteration 120: 0.217710
Cost after iteration 130: 0.212524
Cost after iteration 140: 0.216685
Cost after iteration 150: 0.208888
Cost after iteration 160: 0.199304
Cost after iteration 170: 0.196020
Cost after iteration 180: 0.193798
Cost after iteration 190: 0.190817
Cost after iteration 200: 0.184652
Cost after iteration 210: 0.178050
Cost after iteration 220: 0.177604
Cost after iteration 230: 0.169592
Cost after iteration 240: 0.167180
Cost after iteration 250: 0.162537
Cost after iteration 260: 0.158975
Cost after iteration 270: 0.155302
Cost aft

# mask first

In [30]:
def maskrespondentsanswer(x_pred,maskarr,dic_look):
    x_pred_masked = np.copy(x_pred)
    for i in range(len(maskarr)):
        curmaskkey = maskarr[i]
        cur_loc = dic_look.get(curmaskkey)[0] 
        cur_des = dic_look.get(curmaskkey)[1]
        #start to mask
        for j in range(len(x_pred_masked)):
            for pointer in range(cur_loc,cur_des):
                x_pred_masked[j][pointer] = 0
    print('done')
    return x_pred_masked

In [32]:
# maskarr = ['mood_01','mood_02','big5_01','big5_02','big5_03','big5_04','intrinsic_01','intrinsic_02','intrinsic_03','intrinsic_04','intrinsic_05']

In [33]:
maskarr = ['mood_01','mood_02','big5_01','big5_02','big5_03','big5_04']

In [69]:
x_pred_masked = maskrespondentsanswer(x_pred,maskarr,dic_look)

done


In [70]:
#generate new dataset now

In [92]:
ans = nn.predict(x_pred)

In [297]:
# find ans based on probability 

In [93]:
find_classification_based_on_prob(ans,og_pred)

In [38]:
def find_new_generated_answer(x,y,dic,dic_look,maskarr):
    new_generated_dic_loc = []
    test = []
    countempty = 0
    count = {}
    for i in range(len(x)):
        #if all some genertated answer differet and not empty we record its location
        isvalid = True
        isnew = False
        tmpcount = 0
        for j in range(len(maskarr)):
            cur_mask_key = maskarr[j]
            cur_loc = dic_look.get(cur_mask_key)[0]
            cur_des = dic_look.get(cur_mask_key)[1]
            count_notempty = 0
            #test whether the newly genereated answer is not a valid one
            local_isvalid = False
            local_isnew = False
            x_slice = x[i][cur_loc:cur_des-1]
            y_slice = y[i][cur_loc:cur_des-1]
            if np.array_equal(x_slice,y_slice) == False:
                local_isnew = True
            isnotzero = False
        
            if np.all(x_slice == 0) == False:
                tmpcount+=1
                local_isvalid = True
            else:
                local_isvalid = False
            isvalid = isvalid and local_isvalid
            
            isnew = isnew  or local_isnew
        if isnew == True and isvalid == True: 
            new_generated_dic_loc.append(i)
        
    return new_generated_dic_loc

In [94]:
indexarr = find_new_generated_answer (ans,x_pred,og_pred,dic_look,maskarr)

In [104]:
reverse_dic = create_reverse_dictionary(og_pred)

In [142]:
og_pred.get('lowpower')

{0: 0, 6: 1, 9: 2, 11: 3, 19: 4, 24: 5, 27: 6, 28: 7}

In [145]:
def create_onehoted_text_fixed_classification1(attribu_name,df,km_num):
    df_cur = df[attribu_name]
    attribute_arr = df_cur.values
    
    arr_no_null = processnull(attribute_arr) #create new arr with format fit by NLP(CountVectorizer)
   
    NLP_ARR = NLPDATAPRO(arr_no_null)
    #BEFORE ONEHOT_ENCODING
    #APPLY KMEANS CLUSTERING SINCE TOO MANY DIFFERENT CLASSIFICATION
    KM_TMP = KMeans(km_num)
    KM_TMP.fit2(NLP_ARR)
    labels_arr = KM_TMP.labels
    
    return labels_arr,KM_TMP.centers

In [124]:
def recover_answer_for_generating_data(predict_arr,dic,dic_look):
    reverse_dic = create_reverse_dictionary(og_pred)
    ans_arr = []
    dic_arr = list(dic.keys())
    #recover the predicted
    for i in range(len(predict_arr)):
        tmp_ans_array = {} 
        for j in range(len(dic_arr)):
            cur_dict_key = dic_arr[j]
            cur_loc = dic_look.get(cur_dict_key)[0]
            cur_des = dic_look.get(cur_dict_key)[1]
            tmp_cutting_arr = predict_arr[i][cur_loc:cur_des]
            tmp_cutting_arr2 =  predict_arr[i][cur_loc:cur_des-1]
            cur_reverse_dic_arr = reverse_dic.get(cur_dict_key)
            #first find if all are zero
            if np.all(tmp_cutting_arr2== 0) == True:
                tmp_ans_array[cur_dict_key] = 'NA'
            else:
                #find location
                loc = np.argmax(tmp_cutting_arr2)           
                ans = cur_reverse_dic_arr.get(loc)
                
                tmp_ans_array[cur_dict_key] = ans
        ans_arr.append(tmp_ans_array)
    return ans_arr
    

In [125]:
y = recover_answer_for_generating_data(ans,og_pred,dic_look)

# Convert array to dataframe, and save to csv

In [126]:
te = pd.DataFrame(y)

In [129]:
te.to_csv("./generateddata.csv",index = False,header = True)

# process of answering question in 'Generate data' done

# Answer question in bonus 2

In [306]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr.append(['age',1])
arr.append(['mood_01',2]) #Today I generally feel
arr.append(['mood_02',2])
arr.append(['major',0])
arr.append(['big5_01',2]) #I see myself as: Extraverted, enthusiastic.
arr.append(['big5_02',2]) #I see myself as: Critical, quarrelsome.
arr.append(['big5_03',2]) #I see myself as: Dependable, self-disciplined.
arr.append(['big5_04',2]) #I see myself as: Anxious, easily upset.
arr.append(['big5_05',2]) #I see myself as: Open to new experiences, complex.
arr.append(['big5_06',2]) #I see myself as: Reserved, quiet.
arr.append(['big5_07',2]) #I see myself as: Sympathetic, warm.
arr.append(['big5_08',2]) #I see myself as: Disorganized, careless.
arr.append(['big5_09',2]) #I see myself as: Calm, emotionally stable.
arr.append(['big5_10',2]) #I see myself as: Conventional, uncreative.
arr.append(['highpower',0])
arr.append(['intrinsic_01',2])
arr.append(['intrinsic_02',2])
arr.append(['intrinsic_03',2])
arr.append(['intrinsic_04',2])
arr.append(['intrinsic_05',2])
arr.append(['intrinsic_06',2])
arr.append(['intrinsic_07',2])
arr.append(['intrinsic_08',2])
arr.append(['intrinsic_09',2])
arr.append(['intrinsic_10',2])
arr.append(['intrinsic_11',2])
arr.append(['intrinsic_12',2])
arr.append(['intrinsic_13',2])
arr.append(['intrinsic_14',2])
arr.append(['intrinsic_15',2])
arr.append(['lowpower',0])
arr.append(['mcfiller1',2])
arr.append(['mcfiller2',2])
arr.append(['mcfiller3',2])
arr.append(['mcmost1',2])
arr.append(['mcmost2',2])
arr.append(['mcmost3',2])
arr.append(['mcmost4',2])
arr.append(['mcmost5',2])
arr.append(['mcsome1',2])
arr.append(['mcsome2',2])
arr.append(['mcsome3',2])
arr.append(['mcsome4',2])
arr.append(['mcsome5',2])
arr.append(['mcdv1',2])
arr.append(['mcdv2',2])
arr.append(['pate_01',2])
arr.append(['pate_02',2])
arr.append(['pate_03',2])
arr.append(['pate_04',2])
arr.append(['pate_05',2])
arr.append(['stress_01',2])
arr.append(['stress_02',2])
arr.append(['stress_03',2])
arr.append(['stress_04',2])
arr.append(['nfc_01',2])
arr.append(['nfc_02',2])
arr.append(['nfc_03',2])
arr.append(['nfc_04',2])
arr.append(['nfc_05',2])
arr.append(['nfc_06',2])

In [307]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)

X_train_new = X_train[:2400]
#slice nparr
x_real_train = X_train[:2400]
x_pred = X_train[2400:]
test = createlist_list_string(og_pred)
x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

# find location of where lowpoer sits in the vector

In [309]:
dic_look.get('lowpower')

[213, 222]

# Interpolate section

In [311]:
nn = NeuralNetwork(2000,200,0.1)

In [313]:
nn.fit(X_train_new,X_train_new)

n_x is 374 n_y is  374
Cost after iteration 0: 0.693431
Cost after iteration 10: 0.524898
Cost after iteration 20: 0.365324
Cost after iteration 30: 0.306432
Cost after iteration 40: 0.275571
Cost after iteration 50: 0.259901
Cost after iteration 60: 0.246456
Cost after iteration 70: 0.238691
Cost after iteration 80: 0.234972
Cost after iteration 90: 0.232786
Cost after iteration 100: 0.224932
Cost after iteration 110: 0.219633
Cost after iteration 120: 0.216549
Cost after iteration 130: 0.211898
Cost after iteration 140: 0.210094
Cost after iteration 150: 0.202698
Cost after iteration 160: 0.203223
Cost after iteration 170: 0.199677
Cost after iteration 180: 0.193286
Cost after iteration 190: 0.187340


In [314]:
ans = nn.predict(x_pred)

# Recover Ans 

In [316]:
find_classification_based_on_prob(ans,og_pred)

## len(x_pred)

In [320]:
dic_unanswered = []
for i in range(len(x_pred)):
    if x_pred[i][221] == 0:
        dic_unanswered.append(i)

In [323]:
dic_unanswered

[0,
 1,
 5,
 6,
 7,
 12,
 13,
 14,
 15,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 28,
 31,
 32,
 33,
 34,
 37,
 38,
 39,
 40,
 45,
 50,
 53,
 54,
 57,
 58,
 63,
 65,
 67,
 68,
 72,
 73,
 75,
 77,
 78,
 79,
 80,
 83,
 84,
 90,
 91,
 92,
 93,
 96,
 97,
 98,
 101,
 106,
 107,
 108,
 109,
 110,
 112,
 114,
 115,
 119,
 120,
 121,
 125,
 126,
 128,
 129,
 138,
 139,
 140,
 141,
 142,
 144,
 145,
 147,
 148,
 149,
 151,
 155,
 156,
 157,
 159,
 160,
 162,
 164,
 165,
 166,
 168,
 170,
 171,
 173,
 174,
 175,
 177,
 178,
 179,
 182,
 184,
 185,
 188,
 189,
 190,
 191,
 192,
 193,
 197,
 199,
 200,
 201,
 202,
 204,
 206,
 207,
 208,
 209,
 212,
 216,
 218,
 221,
 223,
 224,
 225,
 228,
 229,
 230,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 243,
 244,
 245,
 250,
 251,
 252,
 253,
 255,
 256,
 258,
 259,
 260,
 261,
 262,
 263,
 266,
 268,
 269,
 270,
 272,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 285,
 287,
 288,
 292,
 294,
 295,
 296,
 297,
 299,
 300,
 303,
 304,


# We have interpolate the unanswed feature of 'lowpower'

In [330]:
ans[0][213:221]

array([0., 0., 0., 0., 0., 0., 0., 1.])

In [331]:
og_pred.get('lowpower')

{0: 0, 6: 1, 9: 2, 11: 3, 19: 4, 24: 5, 27: 6, 28: 7}

# Find out the clustered answer

In [333]:
di = {}
Ans,lab = Bonus_GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df3,'lowpower',di,30)

nan
class_arr of nlp [0, 6, 9, 11, 19, 24, 27, 28]


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [337]:
np.where(lab == 28)

(array([  28,   42,  100,  125,  188,  195,  223,  271,  275,  279,  334,
         370,  372,  413,  419,  484,  529,  590,  641,  781,  849,  852,
         854,  907,  944,  959,  965,  974,  981,  986, 1039, 1041, 1136,
        1142, 1147, 1165, 1172, 1176, 1178, 1180, 1226, 1230, 1251, 1364,
        1365, 1373, 1394, 1421, 1430, 1442, 1450, 1550, 1560, 1572, 1587,
        1592, 1629, 1651, 1660, 1673, 1677, 1706, 1725, 1787, 1790, 1806,
        1807, 1825, 1826, 1840, 1892, 1960, 1969, 1972, 1973, 1978, 2002,
        2005, 2008, 2025, 2040, 2061, 2075, 2078, 2093, 2113, 2147, 2170,
        2185, 2192, 2195, 2201, 2211, 2214, 2216, 2312, 2323, 2340, 2342,
        2352, 2408, 2410, 2417, 2444, 2447, 2451, 2482, 2513, 2522, 2532,
        2596, 2605, 2646, 2665, 2667, 2726, 2781, 2848, 2855, 2869, 2909,
        2921, 2960, 2978, 2991, 2993]),)

# choose one answer which clusterd with the same section

# Finally we use neural network to interpoloate an unanswed column

In [338]:
df3['lowpower'][28]

"I used to work at Applebee's. This is a corporate business so even though we have many managers, they are controlled by the corporate office. I do not agree with the way that things were supposed to be done (according to corporate) and it was very frustrating. They are very strict and have a lot of power not only over my managers, but my fellow employees and myself too."

# Part two assess the quality of these prediction

In [20]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr.append(['age',1])
arr.append(['mood_01',2]) #Today I generally feel
arr.append(['mood_02',2])
arr.append(['major',0])
arr.append(['big5_01',2]) #I see myself as: Extraverted, enthusiastic.
arr.append(['big5_02',2]) #I see myself as: Critical, quarrelsome.
arr.append(['big5_03',2]) #I see myself as: Dependable, self-disciplined.
arr.append(['big5_04',2]) #I see myself as: Anxious, easily upset.
arr.append(['big5_05',2]) #I see myself as: Open to new experiences, complex.
arr.append(['big5_06',2]) #I see myself as: Reserved, quiet.
arr.append(['big5_07',2]) #I see myself as: Sympathetic, warm.
arr.append(['big5_08',2]) #I see myself as: Disorganized, careless.
arr.append(['big5_09',2]) #I see myself as: Calm, emotionally stable.
arr.append(['big5_10',2]) #I see myself as: Conventional, uncreative.
arr.append(['highpower',0])
arr.append(['intrinsic_01',2])
arr.append(['intrinsic_02',2])
arr.append(['intrinsic_03',2])
arr.append(['intrinsic_04',2])
arr.append(['intrinsic_05',2])
arr.append(['intrinsic_06',2])
arr.append(['intrinsic_07',2])
arr.append(['intrinsic_08',2])
arr.append(['intrinsic_09',2])
arr.append(['intrinsic_10',2])
arr.append(['intrinsic_11',2])
arr.append(['intrinsic_12',2])
arr.append(['intrinsic_13',2])
arr.append(['intrinsic_14',2])
arr.append(['intrinsic_15',2])
arr.append(['lowpower',0])
arr.append(['mcfiller1',2])
arr.append(['mcfiller2',2])
arr.append(['mcfiller3',2])
arr.append(['mcmost1',2])
arr.append(['mcmost2',2])
arr.append(['mcmost3',2])
arr.append(['mcmost4',2])
arr.append(['mcmost5',2])
arr.append(['mcsome1',2])
arr.append(['mcsome2',2])
arr.append(['mcsome3',2])
arr.append(['mcsome4',2])
arr.append(['mcsome5',2])
arr.append(['mcdv1',2])
arr.append(['mcdv2',2])
arr.append(['pate_01',2])
arr.append(['pate_02',2])
arr.append(['pate_03',2])
arr.append(['pate_04',2])
arr.append(['pate_05',2])
arr.append(['stress_01',2])
arr.append(['stress_02',2])
arr.append(['stress_03',2])
arr.append(['stress_04',2])
arr.append(['nfc_01',2])
arr.append(['nfc_02',2])
arr.append(['nfc_03',2])
arr.append(['nfc_04',2])
arr.append(['nfc_05',2])
arr.append(['nfc_06',2])
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)

X_train_new = X_train[:2400]
#slice nparr
x_real_train = X_train[:2400]
x_pred = X_train[2400:]
test = createlist_list_string(og_pred)
x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

In [53]:
nn = NeuralNetwork(2000,1000,0.1)

In [54]:
nn.fit(X_train_new,X_train_new)

n_x is 374 n_y is  374
Cost after iteration 0: 0.693403
Cost after iteration 10: 0.450383
Cost after iteration 20: 0.344276
Cost after iteration 30: 0.310549
Cost after iteration 40: 0.272484
Cost after iteration 50: 0.253822
Cost after iteration 60: 0.245454
Cost after iteration 70: 0.236306
Cost after iteration 80: 0.233161
Cost after iteration 90: 0.228167
Cost after iteration 100: 0.228479
Cost after iteration 110: 0.215981
Cost after iteration 120: 0.210314
Cost after iteration 130: 0.205959
Cost after iteration 140: 0.197639
Cost after iteration 150: 0.195877
Cost after iteration 160: 0.191094
Cost after iteration 170: 0.187702
Cost after iteration 180: 0.178968
Cost after iteration 190: 0.178878
Cost after iteration 200: 0.172602
Cost after iteration 210: 0.165447
Cost after iteration 220: 0.161357
Cost after iteration 230: 0.156524
Cost after iteration 240: 0.154319
Cost after iteration 250: 0.148069
Cost after iteration 260: 0.143200
Cost after iteration 270: 0.139292
Cost aft

# Find location of where one hot encoding array represents lowpower

In [55]:
dic_look.get('lowpower')

[213, 222]

# Find the index of respondent who does answer the Natural language feature 'lowpower' question in the testing dataset


In [56]:
indexlist = []
for i in range(len(x_pred)):
    if x_pred[i][221] == 1:
        indexlist.append(i)
len(indexlist)

251

# We Mask their answer manually and utilize neural network to interpolate these answers

In [57]:
x_pred_masked = np.copy(x_pred)
for i in range(len(indexlist)):
    pointer = indexlist[i]
    for j in range(213,222):
        x_pred_masked[pointer][j] = 0

In [59]:
ans = nn.predict(x_pred_masked)

In [61]:
find_classification_based_on_prob(ans,og_pred)

# Count how many masked respond could be interpolated correctly 

In [67]:
count = 0
for i in range(len(indexlist)):
    pointer =  indexlist[i]
    if np.array_equal(ans[pointer][213:221],x_pred[pointer][213:221]):
        count+=1
        
print('  Num of correctly interpolated NLP answer is', count,'\n',' percentage of count/total', count/len(indexlist))

  Num of correctly interpolated NLP answer is 51 
  percentage of count/total 0.20318725099601595


# CREATE FUNCTION FOR CROSS VALIDATION
Assess the accuracy by count how many

In [ ]:
def cross_validation(arr_all_candidate,num_fold,og_pred,size_hidden_layer,iter_num,learning_rate):
    cost = 0
    #create num_fold equal size fold from input nparray
    list_of_fold = np.array_split(arr_all_candidate,num_fold)
    
    #need to append training dataset together, while leave testing set alone
    for i in range(num_fold):
        x_pred = list_of_fold[i]
        
        tmp = 0
        for j in range(0,num_fold):
            if j == i:
                continue
            else:
                
                if tmp == 0:
                    ans = np.asarray(list_of_fold[j])
                    tmp+=1
                
                elif tmp > 0:
                    
                    ans = cro_new_append(ans,list_of_fold[j])

            
        #finished rebuilding training dataset
        #start training neural network
        nn = NeuralNetwork(size_hidden_layer,iter_num,learning_rate) #size of input layer
        
        nn.fit(ans,ans)
        predict_ans = nn.predict(x_pred)
        find_classification_based_on_distance(predict_ans,og_pred)
        #coun
        tmpcost = cost_function(predict_ans,x_pred)
        cost+=tmpcost
        print('#### current cutting position is : ', i, 'current cost is: ',tmpcost)
    print('final avg cost is : ', cost/num_fold)
    return cost/num_fold



def cost_function(x,y):
    #compute the average num of wrongly predicted column as cost function
    total_num = len(x)
    count = 0
    
    for i in range(len(x)):
        tmpcount = 0
        for j in range(len(x[i])):
            if x[i][j]!=y[i][j]:
                tmpcount+=1
        count+=tmpcount
    avg_count = count/total_num
    return avg_count
def cross_valid_append_arr(old,new):
    ans = np.append([old],[new],axis = 0)
    return ans

def cro_new_append(old,new):
    #num_col
    num_col = len(old[0])
    new_ans = []
    for i in range(len(old)):
        new_ans.append(old[i])
    for i in range(len(new)):
        new_ans.append(new[i])
    return np.asarray(new_ans)